In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


sns.set_theme(style="whitegrid")

In [2]:
pip install KNNImputer

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement KNNImputer (from versions: none)
ERROR: No matching distribution found for KNNImputer


In [7]:
from sklearn.impute import KNNImputer


In [5]:
from sklearn.impute import KNNImputer


In [8]:
df = pd.read_csv('Train.csv')
print(df)
df.head()


          ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0          1               D           Flight                    4   
1          2               F           Flight                    4   
2          3               A           Flight                    2   
3          4               B           Flight                    3   
4          5               C           Flight                    2   
...      ...             ...              ...                  ...   
10994  10995               A             Ship                    4   
10995  10996               B             Ship                    4   
10996  10997               C             Ship                    5   
10997  10998               F             Ship                    5   
10998  10999               D             Ship                    2   

       Customer_rating  Cost_of_the_Product  Prior_purchases  \
0                    2                  177                3   
1                    5         

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [9]:
if 'ID' in df.columns:
    df = df.drop('ID',axis=1)
targer_col = 'Reached.on.Time_Y.N'
x = df.drop(targer_col, axis =1)
y = df[targer_col]

print(f"Features Shape: {x.head()}")
print(f"Target Shape: {y.tail()}")

Features Shape:   Warehouse_block Mode_of_Shipment  Customer_care_calls  Customer_rating  \
0               D           Flight                    4                2   
1               F           Flight                    4                5   
2               A           Flight                    2                2   
3               B           Flight                    3                3   
4               C           Flight                    2                2   

   Cost_of_the_Product  Prior_purchases Product_importance Gender  \
0                  177                3                low      F   
1                  216                2                low      M   
2                  183                4                low      M   
3                  176                4             medium      M   
4                  184                3             medium      F   

   Discount_offered  Weight_in_gms  
0                44           1233  
1                59           3088  
2

In [10]:
numeric_features = x.select_dtypes(include=['int64','float64']).columns
categorical_features = x.select_dtypes(include=['object']).columns

print(categorical_features)

print(numeric_features)


Index(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], dtype='object')
Index(['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
       'Prior_purchases', 'Discount_offered', 'Weight_in_gms'],
      dtype='object')


In [11]:
numerical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scalar', StandardScaler())
])

numerical_transformer

,steps,"[('imputer', ...), ('scalar', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,copy,True


In [12]:
categorical_transformer = Pipeline(steps=[
        ('imputer',SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])
categorical_transformer
                        
                            

,steps,"[('imputer', ...), ('onehot', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'constant'
,fill_value,'missing'
,copy,True
,add_indicator,False
,keep_empty_features,False
,categories,'auto'


In [13]:
proprocessor = ColumnTransformer(
    transformers = [
        ('num',numerical_transformer,numeric_features),
        ('cat',categorical_transformer, categorical_features)
    ])
proprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [14]:
pd.set_option('display.max_columns',None)
import warnings

warnings.filterwarnings('ignore')

np.random.seed(42)
n = 1000
df = pd.DataFrame({
    'Age': np.random.normal(35 ,10,n),
    'Income': np.random.lognormal(10,1,n),
    'City': np.random.choice(['New York','London','Paris','Tokyo','Unknown']+[f'City_{i}' for i in range(50)],n),
    'Score': np.random.uniform(1,100,n),
    "Category_A" : np.random.choice(['A','B','C',np.nan],n, p =[0.3,0.3,0.3,0.1]),
    "Feature_x" : np.random.normal(0,1,n),
    "Feature_y" : np.random.normal(0,1,n),
    "Target" : np.random.randint(0,2,n)
})
df.loc[1:10, 'Age']=150 
df.to_csv("Synthetic Data")
df.to_excel("Synthetic-Data.xlsx",index=False)


df

,Age,Income,City,Score,Category_A,Feature_x,Feature_y,Target
0,39.967142,89264.168397,City_24,73.540614,C,0.204759,0.499096,1
1,150.000000,55527.500577,City_25,61.066441,C,0.765878,-1.692271,0
2,150.000000,23379.862874,City_32,67.614295,C,-1.615887,-0.170386,0
3,150.000000,11534.100901,City_5,9.033769,nan,0.299648,-0.812824,0
4,150.000000,44277.118652,City_29,95.197580,A,0.932435,0.828059,0
...,...,...,...,...,...,...,...,...
995,32.188997,64225.155190,City_42,71.757165,A,2.394362,0.069337,1
996,52.976865,21449.974612,City_28,30.041989,B,2.185095,0.862543,1
997,41.408429,9119.090469,City_34,53.021560,nan,-0.686715,-1.694973,0
998,29.288210,18712.238164,City_5,70.085446,A,1.623885,0.820232,1


In [15]:
from sklearn.impute import KNNImputer


knn_imputer = KNNImputer(n_neighbors=5)
df_knn = knn_imputer.fit_transform(df[['Age','Score','Feature_x']])
print("KNN Imputation output shape:", df_knn.shape)
df_knn


KNN Imputation output shape: (1000, 3)


array([[ 39.96714153,  73.540614  ,   0.20475888],
       [150.        ,  61.06644075,   0.76587828],
       [150.        ,  67.61429519,  -1.61588679],
       ...,
       [ 41.40842861,  53.02156043,  -0.68671534],
       [ 29.2882101 ,  70.08544577,   1.62388499],
       [ 40.72582781,  90.12614701,   0.38316819]], shape=(1000, 3))

In [16]:
from sklearn.impute import KNNImputer

cols = ['Age', 'Score', 'Feature_x']

imputer = KNNImputer(n_neighbors=5)

imputed_values = imputer.fit_transform(df[cols].copy())

df_imputed = pd.DataFrame(imputed_values, columns=cols)

print(df_imputed.shape)
df_imputed.head()


(1000, 3)


,Age,Score,Feature_x
0,39.967142,73.540614,0.204759
1,150.000000,61.066441,0.765878
2,150.000000,67.614295,-1.615887
3,150.000000,9.033769,0.299648
4,150.000000,95.197580,0.932435


In [17]:
from sklearn.impute import KNNImputer

cols = ['Age', 'Score', 'Feature_x']

imputer = KNNImputer(n_neighbors=5)

df[cols] = imputer.fit_transform(df[cols])

print("Imputation completed")
df.head()


Imputation completed


,Age,Income,City,Score,Category_A,Feature_x,Feature_y,Target
0,39.967142,89264.168397,City_24,73.540614,C,0.204759,0.499096,1
1,150.000000,55527.500577,City_25,61.066441,C,0.765878,-1.692271,0
2,150.000000,23379.862874,City_32,67.614295,C,-1.615887,-0.170386,0
3,150.000000,11534.100901,City_5,9.033769,nan,0.299648,-0.812824,0
4,150.000000,44277.118652,City_29,95.197580,A,0.932435,0.828059,0


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer

cols = ['Age', 'Score', 'Feature_x']

pipe = Pipeline([
    ("knn_impute", KNNImputer(n_neighbors=5))
])

df_knn = pipe.fit_transform(df[cols])

print(df_knn.shape)


(1000, 3)


In [19]:
from sklearn.impute import KNNImputer

columns_to_impute = ['Age', 'Score', 'Feature_x']

imputer = KNNImputer()

df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

print("Missing values filled using KNN")


Missing values filled using KNN
